In [23]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [25]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/AWI-ESM-1-1-LR/NDJF/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values


array([[[-8.7219028e+00, -8.6877556e+00, -8.4980497e+00, ...,
         -8.5285835e+00, -8.5530157e+00, -8.6450844e+00],
        [-8.3075600e+00, -8.4214182e+00, -8.4099903e+00, ...,
         -8.0717182e+00, -8.0870123e+00, -8.1738434e+00],
        [-7.6313844e+00, -7.8770275e+00, -7.9546103e+00, ...,
         -7.5321035e+00, -7.4825935e+00, -7.4715991e+00],
        ...,
        [-6.1642730e-01, -6.1374807e-01, -6.1168188e-01, ...,
         -6.2799555e-01, -6.2348586e-01, -6.1974651e-01],
        [-4.8045167e-01, -4.7994515e-01, -4.7961330e-01, ...,
         -4.8266780e-01, -4.8180109e-01, -4.8106077e-01],
        [-3.1530568e-01, -3.1525046e-01, -3.1518021e-01, ...,
         -3.1555551e-01, -3.1546474e-01, -3.1537548e-01]],

       [[ 6.4461792e-01,  6.8568319e-01,  7.2813928e-01, ...,
          7.0927215e-01,  6.7028308e-01,  6.4205921e-01],
        [ 6.8644583e-01,  6.7039496e-01,  6.7773068e-01, ...,
          7.8698647e-01,  7.6499051e-01,  7.2159535e-01],
        [ 7.6685715e-01, 

In [18]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [19]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(19840,)


In [20]:
AO_ds.to_netcdf('AWI-ESM-1-1-LR-AOindex-NDJF-Daily-1980-2014.nc')

In [75]:
#AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

In [76]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [77]:
#AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

In [78]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [2]:
zLevData = xr.open_dataset('../../../Data/CMIP6/AWI-ESM-1-1-LR/NDJF/zgNH_NDJFday_AWI-ESM-1-1-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')

In [3]:
z500=zLevData.zg.sel(plev=50000)

In [4]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [5]:
#z145E40N

In [6]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [7]:
#z20E55N

In [8]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [9]:
#z75E55N

In [10]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [11]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [12]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [13]:
EUVal

<xarray.DataArray 'zg' (time: 19840)>
array([-0.66280919, -0.26903376, -0.15079491, ...,  0.04922866,
        0.28706165,  0.19256117])
Coordinates:
  * time       (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 144.4

In [14]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [15]:
EU_ds.to_netcdf('AWI-ESM-1-1-LR-EUindex-NDJF-Daily-1980-2014.nc')